In [3]:
import nltk
import re
import pandas as pd
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer

In [25]:
# create table from txt files
# def create_table():
#     # l is the table and every item of l is the row of the table
#     l = []
#     for i in range(1,1322):
        
#         # temp is the row of table and every item is the cell of table
#         temp = []
#         fdir='news/news'+str(i)+'.txt'
#         with open(fdir,'r') as file:
#             news = file.read()

#             # words tokenization
#             news_words = nltk.word_tokenize(news)
            
#             # preserve english words
#             news_al = [w for w in news_words if re.search('^[a-z]+$', w)]
            
#             # remove stop words
#             news_nonsw = [w for w in news_al if w not in stop_list]
            
#             # stemmer
#             news_stem = [stemmer.stem(w) for w in news_nonsw]
            
#             temp.append('news'+str(i))
#             temp.append(news_stem)
#             l.append(temp)
            
#     # initialize data frame
#     document = pd.DataFrame(l,columns=['news','words'])
#     document['count'] = document['words'].apply(lambda x: Counter(x))
    
#     return document

In [4]:
# create words table from csv file
def create_table():

    news = pd.read_csv('news_for_classification_final.csv')
    news['text'] = news['title'] + news['notes']
    
    # words tokenization
    news['words'] = news['text'].apply(lambda x: nltk.word_tokenize(x))
    
    # preserve english words
    news['char'] = news['words'].apply(lambda x: [w for w in x if re.search('^[a-z]+$', w)])
    
    # remove stop words
    news['nonsw'] = news['char'].apply(lambda x: [w for w in x if w not in stop_list])    
            
    # stemmer
    news['stem'] = news['nonsw'].apply(lambda x: [stemmer.stem(w) for w in x])
            
    # initialize data frame
    document = news[['news','date','title','label','notes','stem']].copy()
    document['count'] = document['stem'].apply(lambda x: Counter(x))
    
    return document

In [5]:
# get word frequency
def count_words(document, key_words):
    
    # initialize the total column
    document['total'] = 0
    
    # count every key words
    for i in range(len(key_words)):
        w = stemmer.stem(key_words[i])
        document[key_words[i]] = document['count'].apply(lambda x: x[w] if w in x.keys() else 0)
        
        # add to total column
        document['total'] += document[key_words[i]]
        
    return document

In [6]:
# get the summary of the results
def summary(document, key_words, n):
    
    # print total
    print('total:')
    news_no = document.nlargest(n, 'total')['news']
    print(news_no)
    print()
    
    # print seperate
    for i in range(len(key_words)):
        print(key_words[i]+':')
        news_no = document.nlargest(n, key_words[i])['news']
        print(news_no)
        print()

In [7]:
# print out the details of the results
def print_out(document, key_words, n):
    
    # print total
    print('total:')
    news_no = document.nlargest(n, 'total')['news']
    for i in range(n):
        news_detail = document[document['news'] == news_no.iloc[i]]
        print('\nresult ',i+1,' words count: ', news_detail.values.tolist()[0][7])
        print('news number: ', news_detail.values.tolist()[0][0])
        print('news date: ', news_detail.values.tolist()[0][1])
        print('news title: ', news_detail.values.tolist()[0][2])
        print('news label: ', news_detail.values.tolist()[0][3])
        print('news notes: ', news_detail.values.tolist()[0][4])
    print()
    
    # print seperate
    for i in range(len(key_words)):
        print(key_words[i]+':')
        news_no = document.nlargest(n, key_words[i])['news']
        for j in range(n):
            news_detail = document[document['news'] == news_no.iloc[j]]
            print('\nresult ',j+1,' words count: ', news_detail.values.tolist()[0][7])
            print('news number: ', news_detail.values.tolist()[0][0])
            print('news date: ', news_detail.values.tolist()[0][1])
            print('news title: ', news_detail.values.tolist()[0][2])
            print('news label: ', news_detail.values.tolist()[0][3])
            print('news notes: ', news_detail.values.tolist()[0][4])
        print()

In [1]:
# main part

# determine the key words, use ',' to split them
# key_words = ['maintenance']
key_words = input().split(',')
key_words

maintenance,oil


['maintenance', 'oil']

In [8]:
# get stop words and stemmer
stop_list = nltk.corpus.stopwords.words('english')
stemmer = nltk.stem.porter.PorterStemmer()

# create words table
document = create_table()
# get word frequency
document = count_words(document, key_words)
# visualize
document.head()

news      date                                              title  label  \
0  news1  2-Jan-19  germany's mothballed wilhelmshaven to restart ...      1   
1  news2  2-Jan-19  italy isab successfully ends 53-day maintenanc...      1   
2  news3  2-Jan-19  italian falconara's planned jan 1 works delaye...      1   
3  news4  2-Jan-19  poland's gdansk operator sees q4 margin rise 1...      0   
4  news5  2-Jan-19  ioc hopeful of iranian funding for cpcl refine...      0   

                                               notes  \
0  notes: hestya energy aims to start operations ...   
1  notes: the isab refinery in southern italy has...   
2  notes: the api refinery in the central italian...   
3  notes: poland's second-largest refiner, grupa ...   
4  notes: state-owned indian oil corp is hopeful ...   

                                                stem  \
0  [germani, mothbal, wilhelmshaven, restart, uni...   
1  [itali, isab, success, end, mainten, sourc, is...   
2  [italian, falconara, plan, jan, work, delay, s...   
3  [poland, gdansk, oper, see, margin, rise, quar...   
4  [ioc, hope, iranian, fund, cpcl, refineri, exp...   

                                               count  total  maintenance  oil  
0  {'germani': 2, 'mothbal': 1, 'wilhelmshaven': ...      1            0    1  
1  {'itali': 2, 'isab': 2, 'success': 2, 'end': 1...      3            3    0  
2  {'italian': 2, 'falconara': 2, 'plan': 2, 'jan...      2            2    0  
3  {'poland': 2, 'gdansk': 1, 'oper': 1, 'see': 1...      0            0    0  
4  {'ioc': 3, 'hope': 2, 'iranian': 2, 'fund': 1,...      1            0    1

In [9]:
# get summary of the results
summary(document, key_words, 5)

total:
204    news205
482    news483
993    news994
67      news68
300    news301
Name: news, dtype: object

maintenance:
719    news720
78      news79
84      news85
199    news200
206    news207
Name: news, dtype: object

oil:
204      news205
993      news994
364      news365
1172    news1173
1214    news1215
Name: news, dtype: object



In [79]:
# get details of the results
print_out(document, key_words, 5)

total:

result  1  words count:  8
news number:  news205
news date:  20-Feb-19
news title:  pdvsa reduces oil supply to cuba for february .
news label:  0
news notes:  seller: pdvsa      buyer: cubametales      country: venezuela      specs & quantity:      one cargo of 505,000 barrels of fuel oil 3% sulfur      one cargo of 500,000 barrels of fuel oil 3% sulfur      one cargo of 500,000 barrels of dco      one cargo of 200,000 barrels of natural gasoline      when: delivery was scheduled for      one cargo of 505,000 barrels of fuel oil 3% sulfur: february 21-23      one cargo of 500,000 barrels of fuel oil 3% sulfur: n/a      one cargo of 500,000 barrels of dco: n/a      one cargo of 200,000 barrels of natural gasoline: february 18-20      ports: cuba      basis:      fuel oil 3%s: hsfo usgc waterbone platts mid flat+ $/barrel. dco: dated brent -$9.50/barrel      gan: platts natural gasoline non -lst non -targa mt bel vieu mo01 mid+6,20 cpg      venezuelan state-owned pdvsa has reduc